# MVVM + Hexagonal in SEVA — Part 2 (View ↔ ViewModel ↔ UseCase)

## What we build in this notebook

We build one small Tkinter app that can:
- Start a demo experiment run
- Poll progress
- Show status text + progress bar

And we build it **step by step** to understand the glue:
1. View with buttons + hooks (UI contract)
2. Glue View ↔ ViewModel
3. Glue ViewModel ↔ UseCase

As in **Part 1**, adapters were introduced there. Here we reuse **`JobRestMock`** to keep focus on MVVM.

In [ ]:
from pathlib import Path
import sys

# Allow notebook execution from docs/ while importing the project package.
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

## 1) Define a View contract and a concrete View (with buttons + hooks)

This view already has buttons, but no business logic inside.
It only exposes hooks (`bind_start`, `bind_poll`) and `render(state)`.

In [ ]:
import tkinter as tk
from tkinter import ttk
from dataclasses import dataclass


@dataclass
class StartVmState:
    # Shared UI state shape used by View + ViewModel.
    status_text: str = "Idle"
    progress_pct: float = 0.0
    is_running: bool = False
    start_clicks: int = 0


class StartViewClassic:
    """Classic View: widgets + hook binding + render, no workflow logic."""

    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("MVVM Demo — Classic View")

        self.status_var = tk.StringVar(value="Idle")
        self.progress_var = tk.DoubleVar(value=0.0)

        outer = ttk.Frame(root, padding=12)
        outer.pack(fill="both", expand=True)

        ttk.Label(outer, text="Experiment status").pack(anchor="w")
        ttk.Label(outer, textvariable=self.status_var).pack(anchor="w", pady=(2, 6))
        ttk.Progressbar(outer, maximum=100, variable=self.progress_var, length=300).pack(anchor="w", pady=(0, 8))

        row = ttk.Frame(outer)
        row.pack(anchor="w")
        self.btn_start = ttk.Button(row, text="Start")
        self.btn_start.pack(side="left", padx=(0, 6))
        self.btn_poll = ttk.Button(row, text="Poll")
        self.btn_poll.pack(side="left")

    def bind_start(self, handler):
        # Hook only: View does not decide WHAT start means.
        self.btn_start.configure(command=handler)

    def bind_poll(self, handler):
        self.btn_poll.configure(command=handler)

    def render(self, state: StartVmState):
        self.status_var.set(state.status_text)
        self.progress_var.set(state.progress_pct)

In [ ]:
# Step 1 assembled app: View + temporary handlers (just to prove hook wiring)
root = tk.Tk()
view = StartViewClassic(root)
state = StartVmState(status_text="Hooks ready", progress_pct=0.0)


def _tmp_start():
    state.status_text = "Start clicked (temporary handler)"
    view.render(state)


def _tmp_poll():
    state.progress_pct = min(100.0, state.progress_pct + 5.0)
    state.status_text = f"Temporary poll... {state.progress_pct:.0f}%"
    view.render(state)


view.bind_start(_tmp_start)
view.bind_poll(_tmp_poll)
view.render(state)
root.mainloop()

## 2) Add ViewModel and glue View ↔ ViewModel

Now button handlers call ViewModel commands.
ViewModel owns UI business rules:
- can start only when not already running
- status transitions
- deterministic progress updates in this intermediate step

In [ ]:
class StartViewModelStepB:
    """ViewModel-only step (no use case yet)."""

    def __init__(self):
        self.state = StartVmState()

    def can_start(self) -> bool:
        return not self.state.is_running

    def on_start_clicked(self):
        if not self.can_start():
            self.state.status_text = "Already running"
            return

        self.state.is_running = True
        self.state.start_clicks += 1
        self.state.progress_pct = 0.0
        self.state.status_text = f"Run #{self.state.start_clicks} started"

    def on_poll_clicked(self):
        if not self.state.is_running:
            self.state.status_text = "Idle (start first)"
            return

        self.state.progress_pct = min(100.0, self.state.progress_pct + 10.0)
        self.state.status_text = f"Running... {self.state.progress_pct:.0f}%"

        if self.state.progress_pct >= 100.0:
            self.state.is_running = False
            self.state.status_text = "Done"

In [ ]:
def assemble_view_with_vm(root: tk.Tk, view: StartViewClassic, vm: StartViewModelStepB):
    """Glue layer for this step: bind UI events to VM commands and re-render."""

    def on_start():
        vm.on_start_clicked()
        view.render(vm.state)

    def on_poll():
        vm.on_poll_clicked()
        view.render(vm.state)

    view.bind_start(on_start)
    view.bind_poll(on_poll)
    view.render(vm.state)

In [ ]:
# Step 2 assembled app: View + ViewModel + glue
root = tk.Tk()
view = StartViewClassic(root)
vm_b = StartViewModelStepB()
assemble_view_with_vm(root, view, vm_b)
root.mainloop()

## 3) Add UseCase and glue ViewModel ↔ UseCase

As in Part 1, adapters are already known. Here we reuse `JobRestMock`.

Important architectural point:
- UseCase depends on `JobPort` (interface),
- We inject `JobRestMock()` as concrete adapter for this tutorial.

In [ ]:
from seva.adapters.job_rest_mock import JobRestMock
from seva.domain.ports import JobPort


class StartAndPollProgressUseCase:
    """UseCase: start a group and poll progress through JobPort."""

    def __init__(self, job_port: JobPort):
        self.job_port = job_port
        self.active_group_id = None

    def start(self) -> str:
        plan = JobRestMock.example_plan()
        group_id, _runs = self.job_port.start_batch(plan)
        self.active_group_id = group_id
        return group_id

    def poll_progress_pct(self) -> float:
        if not self.active_group_id:
            return 0.0

        snapshot = self.job_port.poll_group(self.active_group_id)

        # Simplified extraction for tutorial readability.
        for box in snapshot.get("boxes", {}).values():
            runs = box.get("runs", [])
            if runs:
                return float(runs[0].get("progress_pct", 0.0))
        return 0.0

In [ ]:
class StartViewModel:
    """ViewModel: orchestrates use-case calls + maps results to UI state."""

    def __init__(self, use_case: StartAndPollProgressUseCase):
        self.use_case = use_case
        self.state = StartVmState()

    def can_start(self) -> bool:
        return not self.state.is_running

    def on_start_clicked(self):
        if not self.can_start():
            self.state.status_text = "Already running"
            return

        group_id = self.use_case.start()
        self.state.start_clicks += 1
        self.state.is_running = True
        self.state.progress_pct = 0.0
        self.state.status_text = f"Started group: {group_id}"

    def on_poll_clicked(self):
        if not self.state.is_running:
            self.state.status_text = "Idle (start first)"
            return

        progress = self.use_case.poll_progress_pct()
        self.state.progress_pct = progress

        if progress >= 100.0:
            self.state.is_running = False
            self.state.status_text = "Done"
        else:
            self.state.status_text = f"Running... {progress:.1f}%"

In [ ]:
def assemble_full(root: tk.Tk, view, vm: StartViewModel):
    """Final glue: same binding style, now VM talks to UseCase."""

    def on_start():
        vm.on_start_clicked()
        view.render(vm.state)

    def on_poll():
        vm.on_poll_clicked()
        view.render(vm.state)

    view.bind_start(on_start)
    view.bind_poll(on_poll)
    view.render(vm.state)

### Why `root.after` is not inside ViewModel

Scheduling is kept in the runner/composition function.
That keeps View and ViewModel simpler and easier to test.

In [ ]:
# Step 3 assembled app: View + ViewModel + UseCase + scheduler
root = tk.Tk()
view = StartViewClassic(root)
uc = StartAndPollProgressUseCase(JobRestMock())
vm = StartViewModel(uc)
assemble_full(root, view, vm)


def _tick_poll():
    if vm.state.is_running:
        vm.on_poll_clicked()
        view.render(vm.state)
    root.after(1000, _tick_poll)


_tick_poll()
root.mainloop()

## 4) Swap to a visually distinct (more modern) View

Same VM, same UseCase, same glue style — only View class changes.

In [ ]:
class StartViewModern:
    """More visually distinct view using themed/card-like layout."""

    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("MVVM Demo — Modern View")

        style = ttk.Style(root)
        # Try a modern-ish ttk theme if available.
        if "clam" in style.theme_names():
            style.theme_use("clam")

        style.configure("Card.TFrame", padding=14)
        style.configure("Title.TLabel", font=("TkDefaultFont", 12, "bold"))

        self.status_var = tk.StringVar(value="Idle")
        self.progress_var = tk.DoubleVar(value=0.0)

        shell = ttk.Frame(root, padding=16)
        shell.pack(fill="both", expand=True)

        card = ttk.Frame(shell, style="Card.TFrame")
        card.pack(fill="x")

        ttk.Label(card, text="Experiment Runner", style="Title.TLabel").pack(anchor="w")
        ttk.Label(card, text="Status").pack(anchor="w", pady=(8, 0))
        ttk.Label(card, textvariable=self.status_var).pack(anchor="w", pady=(0, 8))
        ttk.Progressbar(card, maximum=100, variable=self.progress_var, length=320).pack(anchor="w", pady=(0, 10))

        action_row = ttk.Frame(card)
        action_row.pack(anchor="w")
        self.btn_start = ttk.Button(action_row, text="▶ Start")
        self.btn_start.pack(side="left", padx=(0, 8))
        self.btn_poll = ttk.Button(action_row, text="⟳ Poll")
        self.btn_poll.pack(side="left")

    def bind_start(self, handler):
        self.btn_start.configure(command=handler)

    def bind_poll(self, handler):
        self.btn_poll.configure(command=handler)

    def render(self, state: StartVmState):
        self.status_var.set(state.status_text)
        self.progress_var.set(state.progress_pct)

In [ ]:
# Step 4 assembled app: same VM/UseCase, modern View
root = tk.Tk()
modern_view = StartViewModern(root)
uc_m = StartAndPollProgressUseCase(JobRestMock())
vm_m = StartViewModel(uc_m)
assemble_full(root, modern_view, vm_m)


def _tick_poll_modern():
    if vm_m.state.is_running:
        vm_m.on_poll_clicked()
        modern_view.render(vm_m.state)
    root.after(1000, _tick_poll_modern)


_tick_poll_modern()
root.mainloop()